In [0]:
import pandas as pd
import requests
import json

def base_de_datos():
    df_japan = pd.read_csv("https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time")
    df_japan = df_japan[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]  # Campos válidos
    df_japan = df_japan.dropna(subset=['place'])  # Eliminar filas con valores faltantes en la columna 'place'

    estados = ['Alaska', 'California', 'Washington', 'Oregon']
    df_japan = df_japan[df_japan['place'].str.contains('|'.join(estados), case=False)]
    df_japan['localidad'] = df_japan['place']

    df_eu = df_japan.reset_index(drop=True)  # Restablecer el índice a partir de 0
    df_eu['time'] = pd.to_datetime(df_eu['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    df_eu[['latitude','longitude','depth','mag']]=df_eu[['latitude','longitude','depth','mag']].round(1)


    r = requests.get("https://www.jma.go.jp/bosai/quake/data/list.json")  # Obtener datos de la API
    r = r.text  # Texto del cuerpo de la respuesta
    r = json.loads(r)  # Convertir a formato JSON

    quakes = []  # Lista para almacenar los eventos sísmicos

    for item in r:
        if 'cod' in item:
            vars = item['cod'][1:]
            vars = vars.replace('+', ',')
            vars = vars.replace('-', ',')
            vars = vars.replace('/', '')
            vars = vars.split(',')

            if len(vars) >= 3:
                lat = vars[0]
                lon = vars[1]
                dept = vars[2]  # Convertir profundidad a 'km'

                quake = {'time': item['at'], 'latitude': lat, 'longitude': lon, 'depth': dept, 'mag': item['mag'], 'place': item['en_anm'], 'localidad': item['en_anm']}
                quakes.append(quake)
    
    df_japan = pd.DataFrame(quakes)
    #convertimos a float la columna "depth"
    df_japan['depth'] = df_japan['depth'].astype('float64')
    #dividimos por mil para llevar la unidad de medida a KM para mantener la misma en todos los datasets
    df_japan['depth'] = (df_japan['depth'] / 1000)
    df_japan['time'] = pd.to_datetime(df_japan['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    df_japan[['latitude','longitude','depth','mag']]=df_japan[['latitude','longitude','depth','mag']].round(1)

    # Leer el archivo CSV desde la URL
    df_csv = pd.read_csv('https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time')

    # Eliminar filas con valores nulos en la columna 'place'
    df_csv = df_csv.dropna(subset=['place'])

    # Filtrar los registros de México
    df_mexico = df_csv[df_csv['place'].str.contains('Mexico')]

    # Seleccionar las columnas deseadas
    df_mexico = df_mexico[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
    df_mexico['localidad'] = df_mexico['place']
    df_mexico['time'] = pd.to_datetime(df_mexico['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    df_mexico[['latitude','longitude','depth','mag']]=df_mexico[['latitude','longitude','depth','mag']].round(1)

    # Agregar columna 'country' a df_mexico con valor 'mexico'
    df_mexico['country'] = 'mexico'

    # Agregar columna 'country' a df_eu con valor 'EE.UU.'
    df_eu['country'] = 'ee.uu'

    # Agregar columna 'country' a df_japan con valor 'Japan'
    df_japan['country'] = 'japan'

    # Unir los DataFrames utilizando pd.concat()
    df_eu_japan = pd.concat([df_japan, df_eu, df_mexico], axis=0, ignore_index=True)
    df_eu_japan = df_eu_japan.drop(columns=['place'])
    return df_eu_japan

In [0]:
df_eu_jap_mex = base_de_datos()
df_eu_jap_mex

,time,latitude,longitude,depth,mag,localidad,country
0,2023-07-18 20:01:00,36.9,140.9,60.0,3.6,Off the Coast of Fukushima Prefecture,japan
1,2023-07-18 07:33:00,37.4,141.5,40.0,3.7,Off the Coast of Fukushima Prefecture,japan
2,2023-07-17 10:22:00,43.0,142.3,10.0,2.4,"Southern Kamikawa Region, Hokkaido",japan
3,2023-07-17 09:37:00,42.8,143.4,110.0,4.4,"Central Tokachi Region, Hokkaido",japan
4,2023-07-17 09:37:00,42.8,143.4,110.0,4.4,"Central Tokachi Region, Hokkaido",japan
...,...,...,...,...,...,...,...
5239,2023-06-19 02:32:26,31.7,-104.3,6.6,1.9,"57 km S of Whites City, New Mexico",mexico
5240,2023-06-19 02:23:39,31.7,-104.3,6.2,2.6,"58 km S of Whites City, New Mexico",mexico
5241,2023-06-19 01:55:40,31.7,-104.3,6.6,2.1,"58 km S of Whites City, New Mexico",mexico
5242,2023-06-19 01:38:23,31.7,-104.3,7.8,2.1,"57 km S of Whites City, New Mexico",mexico


In [0]:
# Delete row 1
df_eu_jap_mex = df_eu_jap_mex.drop(index=0, inplace=False)
df_eu_jap_mex

,time,latitude,longitude,depth,mag,localidad,country
1,2023-07-18 07:33:00,37.4,141.5,40.0,3.7,Off the Coast of Fukushima Prefecture,japan
2,2023-07-17 10:22:00,43.0,142.3,10.0,2.4,"Southern Kamikawa Region, Hokkaido",japan
3,2023-07-17 09:37:00,42.8,143.4,110.0,4.4,"Central Tokachi Region, Hokkaido",japan
4,2023-07-17 09:37:00,42.8,143.4,110.0,4.4,"Central Tokachi Region, Hokkaido",japan
5,2023-07-17 08:03:00,34.2,135.2,10.0,2.6,Northern Wakayama Prefecture,japan
...,...,...,...,...,...,...,...
5239,2023-06-19 02:32:26,31.7,-104.3,6.6,1.9,"57 km S of Whites City, New Mexico",mexico
5240,2023-06-19 02:23:39,31.7,-104.3,6.2,2.6,"58 km S of Whites City, New Mexico",mexico
5241,2023-06-19 01:55:40,31.7,-104.3,6.6,2.1,"58 km S of Whites City, New Mexico",mexico
5242,2023-06-19 01:38:23,31.7,-104.3,7.8,2.1,"57 km S of Whites City, New Mexico",mexico


In [0]:
import pandas as pd
import requests
import json

# Función para obtener el último sismo
def get_last_quake():
    global df_eu_jap_mex
    countries = {'ee.uu': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time',
                 'japan': 'https://www.jma.go.jp/bosai/quake/data/list.json',
                'mexico': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time'}
    
    for country in countries:
        quake = get_quake_by_country(country, countries[country])
        df_eu_jap_mex = save(quake)
        df_eu_jap_mex_tr = transformaciones(df_eu_jap_mex)
    return df_eu_jap_mex_tr

# Función para obtener el sismo por país
def get_quake_by_country(country, url):
    if country == 'ee.uu':
        df = pd.read_csv(url)
        df = df.dropna(subset=['place'])
        df = df[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
        estados = ['Alaska', 'California', 'Washington', 'Oregon']
        df = df[df['place'].str.contains('|'.join(estados), case=False)]
        df['localidad'] = df['place']
        df['country'] = 'ee.uu'
        df = df.drop(columns=['place'])
        quake = df.head(1)
        # Formato correcto a la columna de fechas
        quake['time'] = pd.to_datetime(quake['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
        #Redondear las columnas 
        quake[['latitude','longitude','depth','mag']]=quake[['latitude','longitude','depth','mag']].round(1)
        return quake
    
    elif country == 'japan':
        r = requests.get(url)
        r = r.text
        r = json.loads(r)
        r = r[0]
        vars = r['cod'][1:]
        vars = vars.replace('+', ',')
        vars = vars.replace('-', ',')
        vars = vars.replace('/', '')
        vars = vars.split(',')
        lat = vars[0]
        lon = vars[1]
        dept = vars[2]
        quake = {'time': r['at'], 'latitude': lat, 'longitude': lon, 'depth': dept, 'mag': r['mag'],'country': 'japon', 'localidad': r['en_anm']}
        quake = pd.DataFrame([quake])
        #convertimos a float la columna "depth"
        quake['depth'] = quake['depth'].astype('float64')
        #dividimos por mil para llevar la unidad de medida a KM para mantener la misma en todos los datasets
        quake['depth'] = (quake['depth'] / 1000)
        # Formato correcto a la columna de fechas
        quake['time'] = pd.to_datetime(quake['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
        #Redondear las columnas 
        quake[['latitude','longitude','depth','mag']]=quake[['latitude','longitude','depth','mag']].round(1)
        return quake
    
    elif country == 'mexico':
        # Leer el archivo CSV desde la URL
        df_csv = pd.read_csv('https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time')
        # Eliminar filas con valores nulos en la columna 'place'
        df_csv = df_csv.dropna(subset=['place'])
        # Filtrar los registros de México
        df_mexico = df_csv[df_csv['place'].str.contains('Mexico')]
        # Seleccionar las columnas deseadas
        df_mexico = df_mexico[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
        df_mexico['localidad'] = df_mexico['place']
        df_mexico['country'] = 'mexico'
        df_mexico = df_mexico.drop(columns=['place'])
        quake = df_mexico.head(1)
        # Formato correcto a la columna de fechas
        quake['time'] = pd.to_datetime(quake['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
        #Redondear las columnas 
        quake[['latitude','longitude','depth','mag']]=quake[['latitude','longitude','depth','mag']].round(1)
        return quake

# Función para guardar el sismo en el DataFrame
def save(quake):
    global df_eu_jap_mex
    df_eu_jap_mex = pd.concat([df_eu_jap_mex, quake], axis=0, ignore_index=True)
    # Drop duplicates
    df_eu_jap_mex = df_eu_jap_mex.drop_duplicates()
    return df_eu_jap_mex


In [0]:
def transformaciones(df):

    # Convertir la columna "Magnitud" y "prrofundidad" a tipo float si es necesario
    df['mag'] = df['mag'].astype(float)
    df['depth'] = df['depth'].astype(float)  # Convertir a tipo float

    # Aplicar la función a cada fila de la columna "Profundidad" y crear la nueva columna "peligrosidad"
    df['Clasificación Profundidad'] = df['depth'].apply(clasificar_profundidad)

    # Aplicar la función a cada fila de la columna "Magnitud" y crear la nueva columna "peligrosidad"
    df['Clasificación Magnitud'] = df['mag'].apply(determinar_peligrosidad)
    
    df = df.drop_duplicates().reset_index(drop=True)
    
    # Renombrar columnas
    df = df.rename(columns={'country': 'país', 'depth': 'profundidad', 'latitude': 'latitud', 'localidad': 'localidad', 'longitude': 'longitud', 'mag': 'magnitud', 'time': 'fecha'})

    # Ordenar columnas
    column_order = ['fecha', 'longitud', 'latitud', 'país', 'localidad' ,'profundidad','Clasificación Profundidad','magnitud','Clasificación Magnitud']
    df = df[column_order] 

    #Todo el dataset a minuscula
    df=df.applymap(lambda x: x.lower() if isinstance(x,str) else x)

    return df

In [0]:
# Definir la función para clasificar la peligrosidad y profundidad 
def determinar_peligrosidad(magnitud):
    if magnitud < 2.0:
        return 'Micro Magnitud'
    elif magnitud < 3.0:
        return 'Menor Magnitud'
    elif magnitud < 4.0:
        return 'Ligera Magnitud'
    elif magnitud < 5.0:
        return 'Moderada Magnitud'
    elif magnitud < 6.0:
        return 'Fuerte Magnitud'
    elif magnitud < 7.0:
        return 'Mayor Magnitud'
    elif magnitud < 8.0:
        return 'Gran Magnitud'
    else:
        return 'Magnitud Épica'
    
def clasificar_profundidad(profundidad):

    if profundidad <= 10:
        return "Superficial"
    elif profundidad > 10 and profundidad <= 30:
        return"Media"
    elif profundidad > 30:
        return"Profundo"


In [0]:
import pandas as pd

# Definir las columnas del DataFrame vacío
columnas = ['fecha', 'longitud', 'latitud', 'país', 'localidad', 'profundidad',
            'Clasificación Profundidad', 'magnitud', 'Clasificación Magnitud']

# Crear el DataFrame vacío
df_vacio = pd.DataFrame(columns=columnas)

In [0]:
def main():
    global df_vacio
    df_referencia = get_last_quake()

    # Ultimo dato de japon
    r = requests.get("https://www.jma.go.jp/bosai/quake/data/list.json")
    r = r.text
    r = json.loads(r)
    r = r[0]
    vars = r['cod'][1:]
    vars = vars.replace('+', ',')
    vars = vars.replace('-', ',')
    vars = vars.replace('/', '')
    vars = vars.split(',')
    lat = vars[0]
    lon = vars[1]
    dept = vars[2]
    quake = {'time': r['at'], 'latitude': lat, 'longitude': lon, 'depth': dept, 'mag': r['mag'], 'localidad': r['en_anm'], 'country': 'japon'}
    Japon = pd.DataFrame([quake])
    #convertimos a float la columna "depth"
    Japon['depth'] = Japon['depth'].astype('float64')
    #dividimos por mil para llevar la unidad de medida a KM para mantener la misma en todos los datasets
    Japon['depth'] = (Japon['depth'] / 1000)
    # Formato correcto a la columna de fechas
    Japon['time'] = pd.to_datetime(Japon['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    Japon[['latitude','longitude','depth','mag']]=Japon[['latitude','longitude','depth','mag']].round(1)

    # Ultimo dato de estados unidos
    df = pd.read_csv('https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time')
    df = df.dropna(subset=['place'])
    df = df[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
    estados = ['Alaska', 'California', 'Washington', 'Oregon']
    df = df[df['place'].str.contains('|'.join(estados), case=False)]
    df['localidad'] = df['place']
    df['country'] = 'ee.uu'
    df = df.drop(columns=['place'])
    eeuu = df.head(1)
    # Formato correcto a la columna de fechas
    eeuu['time'] = pd.to_datetime(eeuu['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    eeuu[['latitude','longitude','depth','mag']]=eeuu[['latitude','longitude','depth','mag']].round(1)
    

    # Ultimo dato de mexico
    df = pd.read_csv('https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time')
    # Eliminar filas con valores nulos en la columna 'place'
    df = df.dropna(subset=['place'])
    # Filtrar los registros de México
    df_mexico = df[df['place'].str.contains('Mexico')]
    # Seleccionar las columnas deseadas
    df_mexico = df_mexico[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
    df_mexico['localidad'] = df_mexico['place']
    df_mexico['country'] = 'mexico'
    df_mexico = df_mexico.drop(columns=['place'])
    mexico = df_mexico.head(1)
    # Formato correcto a la columna de fechas
    mexico['time'] = pd.to_datetime(mexico['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    #Redondear las columnas 
    mexico[['latitude','longitude','depth','mag']]=mexico[['latitude','longitude','depth','mag']].round(1)

    # Concatenar los tres conjuntos de datos
    df_combinado = pd.concat([Japon, eeuu, mexico], ignore_index=True)
    df_combinado = transformaciones(df_combinado)

    df_referencia = pd.concat([df_referencia, df_combinado])
    # Aplico el modelo de Machine Learning
    df_referencia=etiquetado(df_referencia)
    # Actualizar el DataFrame de referencia con las filas faltantes
    df_referencia = df_referencia.drop_duplicates()
    # Aplicar la función 'enviar_notificacion' a la primera fila si no está presente en 'df_vacio'
    if df_vacio.empty:
        enviar_notificacion(df_referencia.iloc[-1])
        df_vacio = pd.concat([df_vacio, df_referencia.iloc[[-1]]], ignore_index=True)

        #Verificar si la última fila de 'df_referencia' está presente en 'df_vacio'
    elif not df_vacio.empty:
        fila_presente = df_vacio.iloc[-1].equals(df_referencia.iloc[-1])

        if not fila_presente:
            enviar_notificacion(df_referencia.iloc[-1])
            # Agregar la primera fila a 'df_vacio' si no está presente
            df_vacio = pd.concat([df_vacio, df_referencia.iloc[[-1]]], ignore_index=True)
        elif fila_presente:
            print("La fila ya está presente en df_vacio. No se enviará la notificación.")
        
    return df_referencia

In [0]:
%pip install python-telegram-bot

Python interpreter will be restarted.
  Using cached python_telegram_bot-20.4-py3-none-any.whl (549 kB)
  Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached httpcore-0.17.3-py3-none-any.whl (74 kB)
  Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached exceptiongroup-1.1.2-py3-none-any.whl (14 kB)
Python interpreter will be restarted.


In [0]:
import pandas as pd
from telegram import Bot

def enviar_notificacion(record):
    # Configurar el token del bot de Telegram
    telegram_token = '6150807593:AAELUrhl2gy62lscxmxGipLSg6pw_l4brMw'
    id_  ='-994506464'
   
    # Extraer detalles del terremoto
    Pais = record['país']
    Magnitud = record['magnitud']
    Latitud = record['latitud']
    Longitud = record['longitud']
    Profundidad = record['profundidad']
    Etiquetas = record['etiquetas']
    Ubicacion = record['localidad']
    if Pais == 'usa':
        emoji_pais = "🇺🇸"
    elif Pais == 'mexico':
        emoji_pais = "🇲🇽"
    elif Pais == 'japon':
        emoji_pais = "🇯🇵"
    else:
        emoji_pais = ""
    
    etiqueta = record['etiquetas']
    if etiqueta == 'leve':
        mensaje = "🌱🌞 Te recomendamos revisar las conexiones de luz, gas y agua para asegurarte de su integridad."
    elif etiqueta == 'alto':
        mensaje = f"⚠️🔥 Si es posible, evacua el edificio y dirígete a un área abierta lejos de objetos que puedan caer."
    elif etiqueta == 'medio':
        mensaje = "🏠 Para tu seguridad, es recomendable resguardarte debajo de una mesa o la cama hasta que el sismo termine."
    else:
        mensaje = "Se ha detectado un sismo. Mantén la calma y sigue las indicaciones de seguridad local."

    # Mensaje de notificación
    message_text = f"🚨 ¡ALERTA DE SISMO EN {Pais.upper()}!\n\n" \
                f"Ubicacion {Ubicacion.upper()} \n" \
                f"🔺Nivel: {etiqueta.capitalize()}🔺\n" \
                f"{mensaje}\n\n" \
                f"Por favor, mantente a salvo y sigue las indicaciones de las autoridades locales."
    send_text = 'https://api.telegram.org/bot' + telegram_token + '/sendMessage?chat_id=' + id_ + '&parse_mode=Markdown&text=' + message_text

    response = requests.get(send_text)

    print('Notificación de Telegram enviada')

In [0]:
import numpy as np
from sklearn.cluster import KMeans

def etiquetado(df):
  # Separar los campos que se utilizarán para ejecutar el modelo
    X = df[['profundidad', 'magnitud']]

    # Seleccionar el modelo K-means
    kmeans = KMeans(n_clusters=4, random_state=0)

    # Entrenar el modelo
    kmeans.fit(X)

    # Obtener las etiquetas
    etiquetas = kmeans.labels_

    # Asignar etiquetas a las categorías 'leve', 'medio' y 'alto'
    df['etiquetas'] = np.where(etiquetas == 0, 'leve',
                                np.where(etiquetas == 1, 'medio',
                                        np.where(etiquetas == 2, 'alto', '')))
    return df

In [0]:
import time

def run_main_every_60_seconds():
    while True:
        df_referencia = main()
        # Puedes realizar cualquier operación adicional con df_referencia aquí
        time.sleep(60)

run_main_every_60_seconds()

<command-1348473818977254>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quake['time'] = pd.to_datetime(quake['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
<command-1348473818977254>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quake[['latitude','longitude','depth','mag']]=quake[['latitude','longitude','depth','mag']].round(1)
<command-1348473818977254>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Notificación de Telegram enviada
